In [ ]:
import astropy.units as u
import numpy as np
import os

os.environ["PIXEDFIT_HOME"] = "/nvme/scratch/work/tharvey/piXedfit/"
from EXPANSE import ResolvedGalaxy, ResolvedGalaxies
from matplotlib import pyplot as plt
import glob
from scipy import signal
from scipy.interpolate import interp1d
from scipy.stats import binned_statistic
# Change dpi to make plots larger

plt.rcParams["figure.dpi"] = 100

# Disable tex in matplotlib

plt.rcParams["text.usetex"] = False

# %matplotlib inline

In [ ]:
galaxies = ResolvedGalaxies(
    ResolvedGalaxy.init_all_field_from_h5("JOF_psfmatched", n_jobs = 6)
)

In [ ]:
a = np.ones_like(galaxies, dtype=bool)
a[10] = False
a[112] = False
print(a)


In [ ]:
table = galaxies.save_to_fits(overwrite=False, save=False)


In [ ]:
table

## Resolved Mass vs. Integrated Mass

In [ ]:
markers = ["o", "s", "D", "^", "v", "<", ">", "p", "P", "*", "X", "d", "H", "h", "+", "x", "|", "_", ".", ","]
bagpipes_runs = ["photoz_lognorm", "photoz_delayed", "photoz_dpl", "photoz_continuity"]

fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi = 200, tight_layout = True, facecolor = 'white')
norm = plt.Normalize( vmin = 4.5, vmax = 10.5)

for marker, bagpipes_run in zip(markers, bagpipes_runs):
    galaxies.comparison_plot(bagpipes_run, "CNST_SFH_RESOLVED_P", label = False, markersize = 4, 
                            markeredgecolor = 'black', markeredgewidth = 0.5, elinewidth = 1, n_jobs = 1, 
                            color_by='redshift', ax = ax, marker = marker, fig = fig, add_colorbar = True if bagpipes_run == "photoz_lognorm" else False, norm = norm) 

# Plot 0.5 dex offset line
xlim = ax.get_xlim()
ylim = ax.get_ylim()

ax.plot(xlim, np.array(ylim)+0.5, color='black', linestyle='--', linewidth=1)
# Label the line with angle text near the top of the line
ax.text(9.35, 9.95, r'0.5 dex', rotation=45, fontsize=8, color='black', ha = 'right', va = 'top')

ax.plot(xlim, np.array(ylim)+1.0, color = 'black', linestyle = '--', linewidth = 1, alpha = 0.6)
ax.text(8.85, 9.95, r'1.0 dex', rotation=45, fontsize=8, color='black', ha = 'right', va = 'top')

# Fake a legend with the markers
points = []
for marker, bagpipes_run in zip(markers, bagpipes_runs):
    point = ax.plot([], [], marker = marker, color = 'black', label = bagpipes_run)
    points.append(point[0])

ax.legend(handles = points, loc = 'upper left', fontsize = 8, title = 'Bagpipes Run')

ax.set_xlabel(r'Integrated Stellar Mass [$\log_{10}(M_{\odot})$]')
ax.set_ylabel(r'Resolved Stellar Mass [$\log_{10}(M_{\odot})$]')

fig.savefig('../plots/resolved_mass_comparison.pdf', dpi = 200, facecolor = 'white', bbox_inches = 'tight')

## Mass Discrepancy as a Function of sSFR

In [ ]:
# use the table of results this time - loop over each SFH history again, plot sSFR vs delta mass. Propogate errors through the calculation.

fig, ax = plt.subplots(1, 1, figsize=(6, 3.5), dpi = 200, tight_layout = True, facecolor = 'white')

plot_dex_diff = True

delta_masses = {}

for marker, bagpipes_run in zip(markers, bagpipes_runs):
    if bagpipes_run == 'photoz_dpl':
        # Something weird going on with this run, skip it for now
        continue
    delta_mass = table['CNST_SFH_RESOLVED_P_resolved_mass'][:, 1] - table[f'{bagpipes_run}_stellar_mass_50']

    delta_masses[bagpipes_run] = delta_mass

    upper_error_resolved = table['CNST_SFH_RESOLVED_P_resolved_mass'][:, 2] - table['CNST_SFH_RESOLVED_P_resolved_mass'][:, 1]
    lower_error_resolved = table['CNST_SFH_RESOLVED_P_resolved_mass'][:, 1] - table['CNST_SFH_RESOLVED_P_resolved_mass'][:, 0]

    upper_error_bagpipes = table[f'{bagpipes_run}_stellar_mass_84'] - table[f'{bagpipes_run}_stellar_mass_50']
    lower_error_bagpipes = table[f'{bagpipes_run}_stellar_mass_50'] - table[f'{bagpipes_run}_stellar_mass_16']

    # Average the errors
    error_resolved = (upper_error_resolved + lower_error_resolved) / 2
    error_bagpipes = (upper_error_bagpipes + lower_error_bagpipes) / 2

    # Propogate errors
    error = np.sqrt(error_resolved**2 + error_bagpipes**2)


    if not plot_dex_diff:
        # Instead plot ratio of resolved mass to bagpipes mass
        delta_mass =  10**table[f'{bagpipes_run}_stellar_mass_50']/10**table['CNST_SFH_RESOLVED_P_resolved_mass'][:, 1] 
        error = np.abs(delta_mass) * np.sqrt((error / table[f'{bagpipes_run}_stellar_mass_50'])**2 + (error / table['CNST_SFH_RESOLVED_P_resolved_mass'][:, 1])**2)



    yerr = error #placeholder for now

    xerr_lower = table[f'{bagpipes_run}_ssfr_10myr_50'] - table[f'{bagpipes_run}_ssfr_10myr_16']
    xerr_upper = table[f'{bagpipes_run}_ssfr_10myr_84'] - table[f'{bagpipes_run}_ssfr_10myr_50']

    xerr = np.array([xerr_lower, xerr_upper])
    xerr[xerr < 0] = 0

    ax.errorbar(table[f'{bagpipes_run}_ssfr_10myr_50'], delta_mass, yerr = yerr, xerr=xerr, marker='none', linestyle='none', alpha = 0.3, zorder=5)
    ax.scatter(table[f'{bagpipes_run}_ssfr_10myr_50'], delta_mass, marker=marker, s=25, edgecolors='black', linewidth=0.5, zorder=10)

# Fix the x-axis limits
ax.set_xlim(ax.get_xlim())
ax.set_ylim(ax.get_ylim())

# Make a dummy legend
points = []
for marker, bagpipes_run in zip(markers, bagpipes_runs):
    point = ax.plot([], [], marker = marker, color = 'black', label = bagpipes_run)
    points.append(point[0])



ax.set_xlabel(r'Specific Star Formation Rate [$\log_{10}(\mathrm{yr}^{-1})$]')
if plot_dex_diff:
    ax.set_ylabel(r'Stellar Mass Offset [$\log_{10}(M_{\odot})$]')
else:
    ax.set_ylabel(r'Integrated / Resolved Stellar Mass')
# Plot 1:1, 0.5 and 1 dex lines
xlim = ax.get_xlim()


ax.plot(xlim, [0, 0], color='black', linestyle='--', linewidth=1)

if plot_dex_diff:
    ax.plot(xlim, np.array([0, 0])+0.5, color='black', linestyle='--', linewidth=1)
    ax.text(-9.3, 0.62, r'0.5 dex', rotation=0, fontsize=8, color='black', ha = 'left', va = 'top')

    ax.plot(xlim, np.array([0, 0])+1.0, color = 'black', linestyle = '--', linewidth = 1, alpha = 0.6)
    ax.text(-9.3, 1.12, r'1.0 dex', rotation=0, fontsize=8, color='black', ha = 'left', va = 'top')
else:
    # Plot 1:1 line
    ax.plot(xlim, [1, 1], color='black', linestyle='--', linewidth=1)
    ax.text(-9.3, 1, r'1:1', rotation=0, fontsize=8, color='black', ha = 'left', va = 'bottom')


if not plot_dex_diff:
    # log scale for y-axis
    ax.set_ylim(0.001, 2)

    #ax.set_yscale('log')

ax.legend(handles = points, loc = 'upper left', fontsize = 8, title = 'Bagpipes Run', frameon = False)


if plot_dex_diff:
    fig.savefig('../plots/ssfr_vs_mass_offset.pdf', dpi = 200, facecolor = 'white', bbox_inches = 'tight')



In [ ]:
# select photoz_lognorm which have delta_masses > 0.2 dex

select_pos = np.zeros(len(galaxies), dtype=bool)

for key in delta_masses.keys():
    select_pos = select_pos | (delta_masses[key] > 0.2)


print(np.sum(select_pos))

galaxy_ids = table['galaxy_id'][select_pos]

galaxies_outshined = ResolvedGalaxies([galaxy for galaxy in galaxies if galaxy.galaxy_id in galaxy_ids])


In [ ]:
for galaxy in galaxies_outshined:
    galaxy.pixedfit_processing(gal_region_use="detection", overwrite=False)
    galaxy.pixedfit_binning(save_out = True, SNR_reqs=4, Dmin_bin=1, redc_chi2_limit=100.0, del_r=1.0, overwrite=True, name_out="pixedfit_nomin")
    

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi = 200, tight_layout = True, facecolor = 'white')
tot_bins = 0
for galaxy in galaxies_outshined:
    nbins = galaxy.get_number_of_bins()
    new_nbins = len(np.unique(galaxy.pixedfit_nomin_map)) - 1
    plt.scatter(nbins, new_nbins)
    tot_bins += new_nbins
    '''
    figg, axx = plt.subplots(2, 1, figsize=(5, 5), dpi = 200, tight_layout = True, facecolor = 'white')
    # Plot old and new bin maps side by side - make colormap using nipy_spectral_r with different colors for each integer bin
    
    cmap = plt.cm.nipy_spectral_r
    cmap.set_under('white')
    cmap.set_bad('white')
    # Plot old bin map
    axx[0].imshow(galaxy.pixedfit_map, cmap=cmap, vmin=0, vmax=nbins)
    axx[0].set_title('Old Bin Map')
    # Plot new bin map
    axx[1].imshow(galaxy.pixedfit_nomin_map, cmap=cmap, vmin=0, vmax=new_nbins)
    axx[1].set_title('New Bin Map')

    plt.show()
    '''
    #galaxy.pixedfit_plot_binmap()

plt.plot([0, 40], [0, 40], color='black', linestyle='--', linewidth=1)

ax.set_xlabel('Old Bin Number')
ax.set_ylabel('New Bin Number')
print(f'Old total_bins {galaxies_outshined.total_number_of_bins()}')
print(f'Total bins: {tot_bins}')


